In [2]:
import numpy as np
import sympy as sp
from plotly import graph_objects as go
import script as rt

Прво си измислуваме / дефинираме податоците.

In [6]:
h_slam_points = np.array([[1, 2, 1, 1], [3, 5, 1.4, 1], [4, 7, 2, 1], [5, 8, 2.6, 1], [7, 7, 3, 1]])
transform = rt.hrotation3('x', sp.pi/3) * rt.htranslation3(2.7, 5, 5.8)
transform = np.array(transform, dtype=np.float)
h_sensor_points = (transform @ h_slam_points.T).T

Изврши ја прво оваа ќелија, потоа наредната и гледај го исходот на графикот.

In [7]:
fig = go.FigureWidget()
fig.update_layout(xaxis={'range': [-7, 6]}, yaxis={'range': [1, 13]})
fig.add_trace(go.Scatter3d(x=h_sensor_points[:, 0], y=h_sensor_points[:, 1], z=h_sensor_points[:, 2], mode='markers', name='Сензори',
                           marker={'size': 10, 'symbol': 'x', 'color': np.arange(len(h_sensor_points))}))
fig.add_trace(go.Scatter3d(x=h_slam_points[:, 0], y=h_slam_points[:, 1], z=h_slam_points[:, 2], mode='markers', name='SLAM', 
                           marker={'size': 10, 'color': np.arange(len(h_slam_points))}))
fig

FigureWidget({
    'data': [{'marker': {'color': array([0, 1, 2, 3, 4]), 'size': 10, 'symbol': 'x'},
         …

In [8]:
def error(h_sensor_points, h_slam_points, x, y, alpha):
    """
    Функција која ја пресметува грешката, на начин на кој сме ја дефинирале.
    """
    transform = rt.hrotation2(alpha) * rt.htranslation2(x, y)
    transform = np.array(transform, dtype=np.float)
    h_hat_sensor_points = (transform @ h_slam_points.T).T
    e = h_sensor_points - h_hat_sensor_points
    return np.apply_along_axis(lambda x: x.T @ x, 1, e)

def step_gradient(h_sensor_points, h_slam_points, transform, learning_rate):
    """
    Чекор со gradient descent.
    
    h_sensor_points, h_slam_points: се податоците кои ќе ни послужат за пресметка на грешката.
    transform: е матрицата на трансформација чии членови ги бараме во оваа функција
    learning_rate: стапката на учење
    """
    x_g, y_g, z_g, rx_g, ry_g, rz_g = 0, 0, 0, 0, 0, 0
    h_hat_sensor_points = (transform @ h_slam_points.T).T  # чекорот го почнуваме од моменталното предвидување...
    r_vector = h_sensor_points - h_hat_sensor_points  # ...сѐ со цел да ја пресметаме грешката која ќе ни треба понатаму
    # тука ги вадиме грешките по x и y за секоја точка. Користиме хомогени координати
    r_x_vector, r_y_vector, r_z_vector = r_vector[:, 0], r_vector[:, 1], r_vector[:, 2]
    m = len(r_vector)
    for r_x, r_y, r_z, h_sensor_point in zip(r_x_vector, r_y_vector, r_z_vector, h_sensor_points):
        x_g += r_x  # го пресметуваме градиентот по x
        y_g += r_y  # го пресметуваме градиентот по y
        z_g += r_z  # го пресметуваме градиентот по z
        rx_g += -h_sensor_point[2] * r_y + h_sensor_point[1] * r_z    # го пресметуваме градиентот за аголот
        ry_g +=  h_sensor_point[2] * r_x - h_sensor_point[0] * r_z    # го пресметуваме градиентот за аголот
        rz_g += -h_sensor_point[1] * r_x + h_sensor_point[0] * r_y    # го пресметуваме градиентот за аголот
    dx = -learning_rate * x_g / m  # потоа со градиентот откриваме колкав чекор треба да направиме по x
    dy = -learning_rate * y_g / m  # потоа со градиентот откриваме колкав чекор треба да направиме по y
    dz = -learning_rate * z_g / m  # потоа со градиентот откриваме колкав чекор треба да направиме по z
    drx = -learning_rate/10 * rx_g / m   # потоа со градиентот откриваме колкав чекор треба да направиме по rx
    dry = -learning_rate/10 * ry_g / m   # потоа со градиентот откриваме колкав чекор треба да направиме по ry
    drz = -learning_rate/10 * rz_g / m   # потоа со градиентот откриваме колкав чекор треба да направиме по rz
#     dalpha = -learning_rate/10 * alpha_gradient / m   # потоа со градиентот откриваме колкав чекор треба да направиме по alpha
    # претходните 3 чекори по секој степен на слобода ги соединуваме во една трансформациона матрица
    delta_transform = np.array(rt.hrotation3('x', drx) * rt.hrotation3('y', dry) * rt.hrotation3('z', drz) * rt.htranslation3(dx, dy, dz), dtype=np.float)
    # и конечно го правиме чекорот
    transform = delta_transform @ transform
    # на крајот ја враќаме обновената вредност на transform и предвидената вредност за изместените податоци за да ги исцртаме
    return transform, h_hat_sensor_points

def run(fig, h_sensor_points, h_slam_points):
    learning_rate = -0.01
    num_iterations = 10000
    # почнуваме со случајно избрани вредности за x, y, и alpha
    x, y, z, alpha, beta, gamma = 0, 0, 0, 0, 0, 0
    transform = np.array(rt.hrotation3('x', alpha) * rt.htranslation3(x, y, z), dtype=np.float)

    for i in range(num_iterations):
        transform, h_hat_sensor_points = step_gradient(h_sensor_points, h_slam_points, transform, learning_rate)
        # цртаме на секоја стота итерација
        if i % 100 == 0:
            x, y, z = transform[0, 3], transform[1, 3], transform[2, 3]
            with fig.batch_update():
                fig.data[1].x, fig.data[1].y, fig.data[1].z = h_hat_sensor_points[:, 0], h_hat_sensor_points[:, 1], h_hat_sensor_points[:, 2]
                fig.data[1].marker.color = np.arange(len(h_hat_sensor_points))
    return transform

final_transform = run(fig, h_sensor_points, h_slam_points)
final_transform

array([[ 9.99471126e-01,  2.40856260e-03, -3.24294088e-02,
         2.75755998e+00],
       [-2.88670931e-02,  5.24859743e-01, -8.50699090e-01,
        -2.58401124e+00],
       [ 1.49719291e-02,  8.51185320e-01,  5.24651686e-01,
         7.20453011e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [9]:
h_slam_points

array([[1. , 2. , 1. , 1. ],
       [3. , 5. , 1.4, 1. ],
       [4. , 7. , 2. , 1. ],
       [5. , 8. , 2.6, 1. ],
       [7. , 7. , 3. , 1. ]])

In [10]:
h_sensor_points

array([[ 3.7       , -2.38897275,  9.46217783,  1.        ],
       [ 5.7       , -1.23538291, 12.26025404,  1.        ],
       [ 6.7       , -0.75499815, 14.29230485,  1.        ],
       [ 7.7       , -0.77461339, 15.45833025,  1.        ],
       [ 9.7       , -1.62102355, 14.79230485,  1.        ]])

In [11]:
transform

array([[ 1.        ,  0.        ,  0.        ,  2.7       ],
       [ 0.        ,  0.5       , -0.8660254 , -2.52294734],
       [ 0.        ,  0.8660254 ,  0.5       ,  7.23012702],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [15]:
final_transform

array([[ 9.99471126e-01,  2.40856260e-03, -3.24294088e-02,
         2.75755998e+00],
       [-2.88670931e-02,  5.24859743e-01, -8.50699090e-01,
        -2.58401124e+00],
       [ 1.49719291e-02,  8.51185320e-01,  5.24651686e-01,
         7.20453011e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [20]:
(transform @ h_slam_points.T).T

array([[ 3.7       , -2.38897275,  9.46217783,  1.        ],
       [ 5.7       , -1.23538291, 12.26025404,  1.        ],
       [ 6.7       , -0.75499815, 14.29230485,  1.        ],
       [ 7.7       , -0.77461339, 15.45833025,  1.        ],
       [ 9.7       , -1.62102355, 14.79230485,  1.        ]])

In [19]:
(final_transform @ h_slam_points.T).T

array([[ 3.72941883, -2.41385794,  9.44652437,  1.        ],
       [ 5.722615  , -1.23729254, 12.23988486,  1.        ],
       [ 6.70744561, -0.7268596 , 14.27201844,  1.        ],
       [ 7.68986765, -0.7412864 , 15.4529667 ,  1.        ],
       [ 9.67342958, -1.66415997, 14.84158591,  1.        ]])